In [ ]:
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python


In [ ]:
# main_parser.py

import os
import json
import asyncio
import requests
from telethon import TelegramClient, functions
from telethon.tl.types import MessageMediaDocument
import nest_asyncio

# Установка nest_asyncio для корректной работы asyncio в определенных средах
nest_asyncio.apply()

# --- Конфигурация и данные из GitHub Secrets ---
try:
    # API_ID преобразуем в число
    API_ID = int(os.environ['API_ID'])
    API_HASH = os.environ['API_HASH']
    PHONE = os.environ['PHONE_NUMBER']

    # URL Webhook передается из main.yml
    N8N_WEBHOOK_URL = os.environ.get('N8N_WEBHOOK_URL')

    if not N8N_WEBHOOK_URL:
        print("🛑 Внимание: Переменная N8N_WEBHOOK_URL не установлена. Данные будут только сохранены локально.")

except KeyError as e:
    print(f"🛑 Критическая ошибка: Не найдена переменная окружения {e}. Проверьте Secrets в репозитории.")
    exit(1)

# --- Список каналов для парсинга ---
# Используйте ТОЛЬКО короткое имя канала (username) с @
CHANNEL_NAMES = ['@sochitypical', '@officialsochi', '@soch01', '@sochi_news4', '@agproshunin', '@livesochi', '@sochi03', '@sochi_online', '@overheardsochi', '@officialsochi']

# --- Файл для локального сохранения (для отладки) ---
OUTPUT_FILE = 'parsed_messages.json'

async def main():
    # Инициализация клиента Telegram
    client = TelegramClient('my_session', API_ID, API_HASH)
    await client.connect()

    # Проверка авторизации
    if not await client.is_user_authorized():
        print("Ошибка: не удалось авторизоваться. Убедитесь, что файл my_session.session загружен.")
        return

    # Попытка присоединиться ко всем каналам (для получения доступа/entity)
    for channel_name in CHANNEL_NAMES:
        try:
            await client(functions.channels.JoinChannelRequest(channel_name))
            print(f"Успешно присоединились или проверили доступ к: {channel_name}")
        except Exception as e:
            print(f"Не удалось присоединиться/проверить доступ к {channel_name}: {e}")

    # Получение последних 5 сообщений из каждого канала
    results = []
    for channel_name in CHANNEL_NAMES:
        print(f"\nПопытка парсинга канала: {channel_name}")
        try:
            async for msg in client.iter_messages(channel_name, limit=5):
                if msg.text or msg.media:
                    message_data = {
                        'channel': channel_name,
                        'message_id': msg.id,
                        'text': msg.text if msg.text else '',
                        'date': str(msg.date),
                        'has_media': False,
                        'media_type': None,
                        'file_name': None,
                    }

                    if msg.media:
                        message_data['has_media'] = True
                        message_data['media_type'] = type(msg.media).__name__

                        # Попытка получить имя файла для документов
                        if isinstance(msg.media, MessageMediaDocument) and msg.media.document.attributes:
                            for attr in msg.media.document.attributes:
                                if hasattr(attr, 'file_name'):
                                    message_data['file_name'] = attr.file_name
                                    break

                    results.append(message_data)

        except Exception as e:
            print(f"🛑 Критическая ошибка при парсинге канала {channel_name}: {e}")
            continue

    print(f"\n[DEBUG] Всего сообщений собрано: {len(results)}")

    # ------------------------------------------------------------------
    # БЛОК ОТЛАДКИ: Сохранение данных в локальный файл для проверки
    # ------------------------------------------------------------------
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"[DEBUG] ✅ Данные успешно сохранены в файл: {OUTPUT_FILE}")
    except Exception as e:
        print(f"[DEBUG] 🛑 Ошибка при сохранении данных в файл: {e}")

    # ------------------------------------------------------------------
    # ГЛАВНЫЙ БЛОК: Отправка данных в n8n Webhook
    # ------------------------------------------------------------------
    if N8N_WEBHOOK_URL and results:
        print(f"[DEBUG] Используемый URL Webhook: {N8N_WEBHOOK_URL}")
        print(f"Отправка {len(results)} сообщений в Webhook n8n...")
        try:
            response = requests.post(N8N_WEBHOOK_URL, json=results, timeout=30)
            # Проверка статуса ответа
            response.raise_for_status()
            print(f"✅ Webhook успешно отправлен. Статус: {response.status_code}. Ответ: {response.text[:50]}...")
        except requests.exceptions.RequestException as e:
            print(f"🛑 Критическая ошибка при отправке Webhook: {e}")
    elif not N8N_WEBHOOK_URL:
        print("🛑 Webhook не отправлен. Переменная N8N_WEBHOOK_URL не установлена.")
    else:
        print("⚠️ Webhook не отправлен. Парсер не нашел новых сообщений для отправки.")

    # Отключение от Telegram
    await client.disconnect()

if __name__ == '__main__':
    try:
        asyncio.run(main())
    except Exception as e:
        print(f"Критическая ошибка выполнения main: {e}")

Данные успешно отправлены в n8n.
